In [102]:
import pandas as pd
import numpy as np
import random as random
from datetime import datetime
import tensorflow as tf
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn import tree, ensemble
from sklearn import svm, linear_model, neighbors
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

pd.set_option('display.max_columns', 60)

# Process data

In [69]:
# All data handled in this notebook comes from: https://github.com/emreg00/repurpose

drug_disease = pd.read_csv("repurpose/data/drug_disease.dat", sep='\t')
print("drug_disease file original shape: {}".format(drug_disease.shape))

drug_disease_flat = pd.DataFrame([], columns=["Drug", "Disease", "Target"])
for index, row in drug_disease.iterrows():
    triplets = []
    for x in drug_disease.columns[1:]:
        triplets.append([row['Drug'], x, row[x]] )
    df_aux = pd.DataFrame(triplets, columns=["Drug", "Disease", "Target"])
    drug_disease_flat = drug_disease_flat.append(df_aux, ignore_index=True) 
drug_disease_flat.head(3)

drug_disease file original shape: (799, 720)


,Drug,Disease,Target
0,6-aminocaproic acid,AIDS.Related.Opportunistic.Infections,0
1,6-aminocaproic acid,Abdominal.Abscess,0
2,6-aminocaproic acid,Abnormalities..Drug.Induced,0


In [70]:
drug_protein = pd.read_csv("repurpose/data/drug_protein.dat", sep='\t')
print(drug_protein.shape)
print(drug_protein.iloc[:, 1:].to_numpy().max())
drug_protein.head(2)

(1007, 776)
1


,Drug,A2A2V4,A8MPY1,A9UF02,B8DCL9,B8DD61,C1KC03,O00180,O00305,O00459,O00519,O00555,O00591,O14521,O14646,O14649,O14717,O14732,O14764,O14788,O14920,O15111,O15379,O15399,O15438,O15439,O15530,O15554,O25608,O43451,O43497,O43519,O43526,O43570,O43772,O43923,O60391,O60488,O60494,O60658,O60706,O60733,O60840,O60939,O67135,O75311,O75469,O75493,O75762,O75899,O76074,O76082,O90777,O94956,O95069,O95180,O95263,O95298,O95342,O95477,O95665,P00167,P00183,P00325,P00326,P00363,P00367,P00374,P00390,P00439,P00484,P00491,P00492,P00519,P00533,P00582,P00644,P00720,P00722,P00734,P00742,P00747,P00749,P00750,P00797,P00807,P00811,P00813,P00915,P00918,P01008,P01023,P01130,P01138,P01189,P01375,P01579,P01584,P02359,P02585,P02708,P02787,P02818,P02918,P02919,P03176,P03198,P03200,P03366,P03369,P03372,P03886,P04015,P04035,P04040,P04049,P04054,P04070,P04083,P04150,P04156,P04271,P04293,P04626,P04629,P04746,P04818,P05023,P05091,P05093,P05106,P05107,P05108,P05121,P05164,P05193,P05230,P05231,P05412,P05546,P05981,P06149,P06239,P06241,P06276,P06310,P06401,P06478,P06715,P06746,P06818,P06855,P06864,P06971,P07003,P07098,P07101,P07202,P07225,P07327,P07333,P07437,P07451,P07477,P07510,P07550,P07900,P07947,P07998,P08100,P08149,P08172,P08173,P08174,P08183,P08192,P08235,P08253,P08506,P08514,P08519,P08546,P08588,P08620,P08700,P08729,P08908,P08912,P08913,P09038,P09172,P09210,P09211,P09250,P09252,P09455,P09619,P09884,P09917,P0A0K8,P0A0N4,P0A3M5,P0A3M6,P0A4L9,P0A560,P0A5Y6,P0A674,P0A6E4,P0A6J8,P0A749,P0A7J6,P0A7R5,P0A7S3,P0A7S9,P0A7U3,P0A7V3,P0A7V8,P0A7W7,P0A7X3,P0A7Z4,P0A8T7,P0A8V2,P0AC13,P0AD63,P0AD65,P0AD68,P0ADY7,P0AEB2,P0AEB3,P0AES6,P0AFI2,P0AFI5,P0AG59,P0C002,P0C0X1,P0C0X2,P10109,P10253,P10275,P10276,P10415,P10613,P10696,P10721,P10826,P10827,P10828,P10875,P11137,P11229,P11230,P11387,P11388,P11473,P11485,P11511,P12268,P12314,P12319,P12461,P12821,P12931,P13500,P13569,P13631,P13674,P13716,P13922,P13945,P14324,P14410,P14416,P14555,P14625,P14677,P14679,P14735,P14770,P14780,P14867,P15056,P15121,P15144,P15382,P15538,P15555,P15559,P15692,P16066,P16220,P16233,P16234,P16444,P16502,P16860,P17117,P17302,P17405,P17752,P17787,P17948,P18031,P18089,P18314,P18405,P18505,P18507,P18545,P18825,P18858,P19320,P19634,P19793,P19801,P20061,P20062,P20309,P20594,P20618,P20648,P20701,P20711,P20813,P20839,P21397,P21430,P21462,P21554,P21589,P21728,P21817,P21917,P21918,P21964,P22033,P22102,P22303,P22413,P22748,P23141,P23219,P23258,P23280,P23297,P23415,P23416,P23469,P23786,P23921,P23975,P24024,P24046,P24093,P24228,P24298,P24385,P24530,P24752,P25021,P25024,P25100,P25101,P25103,P25815,P26358,P26676,P27169,P27338,P27361,P27487,P27707,P27815,P27907,P27986,P28074,P28221,P28222,P28223,P28335,P28472,P28476,P28482,P28566,P28702,P29034,P29074,P29166,P29218,P29274,P29275,P29317,P29466,P29474,P29972,P30049,P30085,P30273,P30405,P30518,P30531,P30532,P30536,P30542,P30556,P30926,P30939,P30968,P31213,P31350,P31644,P31645,P31749,P31939,P32239,P32297,P32754,P33151,P33402,P33527,P33673,P33765,P34903,P34969,P34972,P34995,P35218,P35219,P35222,P35228,P35348,P35354,P35367,P35368,P35372,P35408,P35462,P35498,P35499,P35610,P35916,P35968,P36544,P36888,P37088,P37231,P37288,P38435,P38489,P38606,P39086,P39900,P41143,P41145,P41180,P41231,P41594,P41595,P41972,P42261,P42262,P42330,P42345,P42574,P42684,P42971,P43088,P43115,P43116,P43119,P43133,P43155,P43166,P43681,P43700,P43702,P44345,P44469,P45059,P45954,P46098,P46925,P47712,P47869,P47870,P47901,P47989,P48039,P48048,P48051,P48061,P48169,P48443,P48544,P48549,P48775,P49019,P49146,P49228,P49286,P49327,P49441,P49721,P49841,P49888,P49902,P49916,P50135,P50406,P50416,P50859,P50870,P51168,P51170,P51172,P51681,P51692,P51787,P51788,P51800,P51843,P51857,P52209,P52647,P52895,P53608,P53848,P54284,P54289,P54710,P54750,P55011,P55017,P56282,P59768,P60174,P60723,P60725,P61073,P61177,P61278,P61889,P62158,P62508,P62580,P62594,P62942,P63316,P64143,P66010,P68344,P68371,P68638,P69905,P72030,P72059,P72161,P72524,P72525,P74872,P76577,P77390,P78334,P78348,P78352,P78559,P80404,P80511,P83223,P87066,P94692,P95029,P965

In [71]:
drug_structure = pd.read_csv("repurpose/data/drug_structure.dat", sep='\t')
print(drug_structure.shape)
print(drug_structure.iloc[:, 1:].to_numpy().max())
drug_structure.head(2)

(1007, 882)
1


,Drug,SUB.0,SUB.1,SUB.2,SUB.3,SUB.4,SUB.5,SUB.6,SUB.7,SUB.8,SUB.9,SUB.10,SUB.11,SUB.12,SUB.13,SUB.14,SUB.15,SUB.16,SUB.17,SUB.18,SUB.19,SUB.20,SUB.21,SUB.22,SUB.23,SUB.24,SUB.25,SUB.26,SUB.27,SUB.28,SUB.29,SUB.30,SUB.31,SUB.32,SUB.33,SUB.34,SUB.35,SUB.36,SUB.37,SUB.38,SUB.39,SUB.40,SUB.41,SUB.42,SUB.43,SUB.44,SUB.45,SUB.46,SUB.47,SUB.48,SUB.49,SUB.50,SUB.51,SUB.52,SUB.53,SUB.54,SUB.55,SUB.56,SUB.57,SUB.58,SUB.59,SUB.60,SUB.61,SUB.62,SUB.63,SUB.64,SUB.65,SUB.66,SUB.67,SUB.68,SUB.69,SUB.70,SUB.71,SUB.72,SUB.73,SUB.74,SUB.75,SUB.76,SUB.77,SUB.78,SUB.79,SUB.80,SUB.81,SUB.82,SUB.83,SUB.84,SUB.85,SUB.86,SUB.87,SUB.88,SUB.89,SUB.90,SUB.91,SUB.92,SUB.93,SUB.94,SUB.95,SUB.96,SUB.97,SUB.98,SUB.99,SUB.100,SUB.101,SUB.102,SUB.103,SUB.104,SUB.105,SUB.106,SUB.107,SUB.108,SUB.109,SUB.110,SUB.111,SUB.112,SUB.113,SUB.114,SUB.115,SUB.116,SUB.117,SUB.118,SUB.119,SUB.120,SUB.121,SUB.122,SUB.123,SUB.124,SUB.125,SUB.126,SUB.127,SUB.128,SUB.129,SUB.130,SUB.131,SUB.132,SUB.133,SUB.134,SUB.135,SUB.136,SUB.137,SUB.138,SUB.139,SUB.140,SUB.141,SUB.142,SUB.143,SUB.144,SUB.145,SUB.146,SUB.147,SUB.148,SUB.149,SUB.150,SUB.151,SUB.152,SUB.153,SUB.154,SUB.155,SUB.156,SUB.157,SUB.158,SUB.159,SUB.160,SUB.161,SUB.162,SUB.163,SUB.164,SUB.165,SUB.166,SUB.167,SUB.168,SUB.169,SUB.170,SUB.171,SUB.172,SUB.173,SUB.174,SUB.175,SUB.176,SUB.177,SUB.178,SUB.179,SUB.180,SUB.181,SUB.182,SUB.183,SUB.184,SUB.185,SUB.186,SUB.187,SUB.188,SUB.189,SUB.190,SUB.191,SUB.192,SUB.193,SUB.194,SUB.195,SUB.196,SUB.197,SUB.198,SUB.199,SUB.200,SUB.201,SUB.202,SUB.203,SUB.204,SUB.205,SUB.206,SUB.207,SUB.208,SUB.209,SUB.210,SUB.211,SUB.212,SUB.213,SUB.214,SUB.215,SUB.216,SUB.217,SUB.218,SUB.219,SUB.220,SUB.221,SUB.222,SUB.223,SUB.224,SUB.225,SUB.226,SUB.227,SUB.228,SUB.229,SUB.230,SUB.231,SUB.232,SUB.233,SUB.234,SUB.235,SUB.236,SUB.237,SUB.238,SUB.239,SUB.240,SUB.241,SUB.242,SUB.243,SUB.244,SUB.245,SUB.246,SUB.247,SUB.248,SUB.249,SUB.250,SUB.251,SUB.252,SUB.253,SUB.254,SUB.255,SUB.256,SUB.257,SUB.258,SUB.259,SUB.260,SUB.261,SUB.262,SUB.263,SUB.264,SUB.265,SUB.266,SUB.267,SUB.268,SUB.269,SUB.270,SUB.271,SUB.272,SUB.273,SUB.274,SUB.275,SUB.276,SUB.277,SUB.278,SUB.279,SUB.280,SUB.281,SUB.282,SUB.283,SUB.284,SUB.285,SUB.286,SUB.287,SUB.288,SUB.289,SUB.290,SUB.291,SUB.292,SUB.293,SUB.294,SUB.295,SUB.296,SUB.297,SUB.298,SUB.299,SUB.300,SUB.301,SUB.302,SUB.303,SUB.304,SUB.305,SUB.306,SUB.307,SUB.308,SUB.309,SUB.310,SUB.311,SUB.312,SUB.313,SUB.314,SUB.315,SUB.316,SUB.317,SUB.318,SUB.319,SUB.320,SUB.321,SUB.322,SUB.323,SUB.324,SUB.325,SUB.326,SUB.327,SUB.328,SUB.329,SUB.330,SUB.331,SUB.332,SUB.333,SUB.334,SUB.335,SUB.336,SUB.337,SUB.338,SUB.339,SUB.340,SUB.341,SUB.342,SUB.343,SUB.344,SUB.345,SUB.346,SUB.347,SUB.348,SUB.349,SUB.350,SUB.351,SUB.352,SUB.353,SUB.354,SUB.355,SUB.356,SUB.357,SUB.358,SUB.359,SUB.360,SUB.361,SUB.362,SUB.363,SUB.364,SUB.365,SUB.366,SUB.367,SUB.368,SUB.369,SUB.370,SUB.371,SUB.372,SUB.373,SUB.374,SUB.375,SUB.376,SUB.377,SUB.378,SUB.379,SUB.380,SUB.381,SUB.382,SUB.383,SUB.384,SUB.385,SUB.386,SUB.387,SUB.388,SUB.389,SUB.390,SUB.391,SUB.392,SUB.393,SUB.394,SUB.395,SUB.396,SUB.397,SUB.398,SUB.399,SUB.400,SUB.401,SUB.402,SUB.403,SUB.404,SUB.405,SUB.406,SUB.407,SUB.408,SUB.409,SUB.410,SUB.411,SUB.412,SUB.413,SUB.414,SUB.415,SUB.416,SUB.417,SUB.418,SUB.419,SUB.420,SUB.421,SUB.422,SUB.423,SUB.424,SUB.425,SUB.426,SUB.427,SUB.428,SUB.429,SUB.430,SUB.431,SUB.432,SUB.433,SUB.434,SUB.435,SUB.436,SUB.437,SUB.438,SUB.439,SUB.440,SUB.441,SUB.442,SUB.443,SUB.444,SUB.445,SUB.446,SUB.447,SUB.448,SUB.449,SUB.450,SUB.451,SUB.452,SUB.453,SUB.454,SUB.455,SUB.456,SUB.457,SUB.458,SUB.459,SUB.460,SUB.461,SUB.462,SUB.463,SUB.464,SUB.465,SUB.466,SUB.467,SUB.468,SUB.469,SUB.470,SUB.471,SUB.472,SUB.473,SUB.474,SUB.475,SUB.476,SUB.477,SUB.478,SUB.479,SUB.480,SUB.481,SUB.482,SUB.483,SUB.484,SUB.485,SUB.486,SUB.487,SUB.488,SUB.489,SUB.490,SUB.491,SUB.492,SUB.493,SUB.494,SUB.495,SUB.496,SUB.497,SUB.498,SUB.499,SUB.500,SUB.501,SUB.502,SUB.503,SUB.504,SUB.505,SUB.506,SUB.507,SUB.508,SUB.509,SUB.510,SUB.511,SUB.512,

In [72]:
drug_sider = pd.read_csv("repurpose/data/drug_sider.dat", sep='\t')
print(drug_sider.shape)
print(drug_sider.iloc[:, 1:].to_numpy().max())
drug_sider.head(2)

(888, 1386)
1


,Drug,abdominal.cramps,abdominal.distention,abdominal.pain,malformations,spontaneous.abortion,missed.abortion,abscess,acanthosis.nigricans,acidosis,renal.tubular.acidosis,acquired.immunodeficiency.syndrome,acute.pancreatitis,adenocarcinoma,adenomas,adjustment.disorder,adrenal.disease,adrenal.insufficiency,agnosia,agoraphobia,agranulocytosis,albuminuria,delirium.tremens,drunk,alcohol.dependence,alkalosis,respiratory.alkalosis,allergic.rhinitis,alopecia,allergic.alveolitis,amblyopia,amenorrhea,hyperaminoaciduria,amnesia,amyloidosis,als,anaphylaxis,anemia,aplastic.anemia,hemolytic.anemia,autoimmune.hemolytic.anemia,hypochromic.anemia,macrocytic.anemia,megaloblastic.anemia,pernicious.anemia,sickle.cell.disease,sideroblastic.anemia,aneurysm,angina.pectoris,Variant.Angina,angioedema,anisocoria,anorexia,anosmia,anxiety,aortic.aneurysm,Aortic.valve.incompetence,aortic.stenosis,aphasia,aphonia,apnea,appendicitis,apraxia,arachnoiditis,arrhythmia,arterial.insufficiency,arteriosclerosis,arteritis,arthralgia,arthritis,gouty.arthritis,rheumatoid.arthritis,septic.arthritis,ascites,aspergillosis,asthenia,asthenopia,asthma,astrocytoma,ataxia,atelectasis,atherosclerotic.vascular.disease,atrial.fibrillation,atrial.flutter,AV.block,back.pain,bacteremia,bacterial.infections,balanitis,skin.benign.neoplasm,benign.prostatic.hyperplasia,biliary.tract.disorder,bipolar.disorder,distended.bladder,bladder.tumors,neurogenic.bladder,blepharitis,blepharoconjunctivitis,blepharoptosis,blepharospasm,blistering,Clotting,platelet.disorder,bone.neoplasm,borderline.personality.disorder,brain.abscess,metabolic.encephalopathy,brain.tumor,Breast.cyst,breast.disorders,Breast.swelling,bronchospasm,bronchiectasis,bronchiolitis,bronchitis,bruit,bruxism,bulimia,bundle.branch.block,bursitis,cachexia,cancer,moniliasis,cutaneous.candidiasis,oral.moniliasis,vaginal.moniliasis,carbuncle,carcinoid.tumors,carcinoma,rectal.carcinoma,basal.cell.carcinoma,cardiac.tamponade,carotid.bruit,carotid.stenosis,carpal.tunnel.syndrome,cataplexy,causalgia,sprue,cellulitis,cerebellar.ataxia,cerebral.embolism,cerebral.hemorrhage,cerebral.infarction,transient.ischemic.attack,cerebral.palsy,cerebrovascular.disorder,neck.pain,cervicitis,cervical.dysplasia,cervical.erosion,chalazion,chest.pain,pleuritic.pain,chest.wall.pain,varicella,choking,cholangitis,sclerosing.cholangitis,cholecystitis,cholelithiasis,chondrodystrophy,choriocarcinoma,chorioretinitis,scarring,cleft.lip,cleft.palate,cluster.headache,coated.tongue,Cognitive.Disorder,cold.intolerance,colitis,ulcerative.colitis,Collagen.Disorders,colonic.polyp,coma,common.cold,Infectious.Diseases,confusion,conjunctivitis,connective.tissue.disorders,constipation,constitutional.symptoms,contracture,convulsions,febrile.seizures,corneal.disorders,corneal.opacity,corneal.ulcer,coronary.artery.disease,Corpus.luteum.cyst,cough,cryptococcosis,Cryptosporidiosis,Cushing.syndrome,cystic.acne,cystitis,cyst,CMV.infection,dacryocystitis,deafness,decreased.libido,decubitus.ulcers,dysphagia,dehydration,delirium,tooth.caries,mental.depression,depressive.reaction,dermatitis,exfoliative.dermatitis,drug.eruption,atopic.dermatitis,contact.dermatitis,fungal.dermatitis,dermatomyositis,dermatophytosis,dermoid.cyst,diabetes.insipidus,diabetes.mellitus,insulin.dependent.diabetes.mellitus,type.2.diabetes,Diabetic.ketoacidosis,diabetic.neuropathy,diabetic.retinopathy,Diaper.rash,diarrhea,diplopia,diverticulitis,dizziness,Trisomy.21,somnolence,abuse,drug.hypersensitivity,patent.ductus.arteriosus,duodenal.ulcer,duodenitis,dwarfism,dysarthria,Infectious.Diarrhea,dyskinesia,dysmenorrhea,Dyspepsia,dyspnea,paroxysmal.nocturnal.dyspnea,dysuria,ear.pain,eating.disorder,ecchymosis,eclampsia,ectropion,eczema,edema,embolism,fat.embolism,encephalitis,encopresis,endocarditis,Endocrine.Disorders,endometrial.hyperplasia,endometriosis,endometritis,enteritis,enterocolitis,enuresis,eosinophilia,Epicondylitis,sebaceous.cyst,toxic.epidermal.necrolysis,epididymitis,epiglottitis,epilepsy,grand.mal,m

In [73]:
drugs_inputs = pd.merge(drug_protein, drug_structure, on="Drug")
print(drugs_inputs.shape)
drugs_inputs.head(2)

(944, 1657)


,Drug,A2A2V4,A8MPY1,A9UF02,B8DCL9,B8DD61,C1KC03,O00180,O00305,O00459,O00519,O00555,O00591,O14521,O14646,O14649,O14717,O14732,O14764,O14788,O14920,O15111,O15379,O15399,O15438,O15439,O15530,O15554,O25608,O43451,O43497,O43519,O43526,O43570,O43772,O43923,O60391,O60488,O60494,O60658,O60706,O60733,O60840,O60939,O67135,O75311,O75469,O75493,O75762,O75899,O76074,O76082,O90777,O94956,O95069,O95180,O95263,O95298,O95342,O95477,O95665,P00167,P00183,P00325,P00326,P00363,P00367,P00374,P00390,P00439,P00484,P00491,P00492,P00519,P00533,P00582,P00644,P00720,P00722,P00734,P00742,P00747,P00749,P00750,P00797,P00807,P00811,P00813,P00915,P00918,P01008,P01023,P01130,P01138,P01189,P01375,P01579,P01584,P02359,P02585,P02708,P02787,P02818,P02918,P02919,P03176,P03198,P03200,P03366,P03369,P03372,P03886,P04015,P04035,P04040,P04049,P04054,P04070,P04083,P04150,P04156,P04271,P04293,P04626,P04629,P04746,P04818,P05023,P05091,P05093,P05106,P05107,P05108,P05121,P05164,P05193,P05230,P05231,P05412,P05546,P05981,P06149,P06239,P06241,P06276,P06310,P06401,P06478,P06715,P06746,P06818,P06855,P06864,P06971,P07003,P07098,P07101,P07202,P07225,P07327,P07333,P07437,P07451,P07477,P07510,P07550,P07900,P07947,P07998,P08100,P08149,P08172,P08173,P08174,P08183,P08192,P08235,P08253,P08506,P08514,P08519,P08546,P08588,P08620,P08700,P08729,P08908,P08912,P08913,P09038,P09172,P09210,P09211,P09250,P09252,P09455,P09619,P09884,P09917,P0A0K8,P0A0N4,P0A3M5,P0A3M6,P0A4L9,P0A560,P0A5Y6,P0A674,P0A6E4,P0A6J8,P0A749,P0A7J6,P0A7R5,P0A7S3,P0A7S9,P0A7U3,P0A7V3,P0A7V8,P0A7W7,P0A7X3,P0A7Z4,P0A8T7,P0A8V2,P0AC13,P0AD63,P0AD65,P0AD68,P0ADY7,P0AEB2,P0AEB3,P0AES6,P0AFI2,P0AFI5,P0AG59,P0C002,P0C0X1,P0C0X2,P10109,P10253,P10275,P10276,P10415,P10613,P10696,P10721,P10826,P10827,P10828,P10875,P11137,P11229,P11230,P11387,P11388,P11473,P11485,P11511,P12268,P12314,P12319,P12461,P12821,P12931,P13500,P13569,P13631,P13674,P13716,P13922,P13945,P14324,P14410,P14416,P14555,P14625,P14677,P14679,P14735,P14770,P14780,P14867,P15056,P15121,P15144,P15382,P15538,P15555,P15559,P15692,P16066,P16220,P16233,P16234,P16444,P16502,P16860,P17117,P17302,P17405,P17752,P17787,P17948,P18031,P18089,P18314,P18405,P18505,P18507,P18545,P18825,P18858,P19320,P19634,P19793,P19801,P20061,P20062,P20309,P20594,P20618,P20648,P20701,P20711,P20813,P20839,P21397,P21430,P21462,P21554,P21589,P21728,P21817,P21917,P21918,P21964,P22033,P22102,P22303,P22413,P22748,P23141,P23219,P23258,P23280,P23297,P23415,P23416,P23469,P23786,P23921,P23975,P24024,P24046,P24093,P24228,P24298,P24385,P24530,P24752,P25021,P25024,P25100,P25101,P25103,P25815,P26358,P26676,P27169,P27338,P27361,P27487,P27707,P27815,P27907,P27986,P28074,P28221,P28222,P28223,P28335,P28472,P28476,P28482,P28566,P28702,P29034,P29074,P29166,P29218,P29274,P29275,P29317,P29466,P29474,P29972,P30049,P30085,P30273,P30405,P30518,P30531,P30532,P30536,P30542,P30556,P30926,P30939,P30968,P31213,P31350,P31644,P31645,P31749,P31939,P32239,P32297,P32754,P33151,P33402,P33527,P33673,P33765,P34903,P34969,P34972,P34995,P35218,P35219,P35222,P35228,P35348,P35354,P35367,P35368,P35372,P35408,P35462,P35498,P35499,P35610,P35916,P35968,P36544,P36888,P37088,P37231,P37288,P38435,P38489,P38606,P39086,P39900,P41143,P41145,P41180,P41231,P41594,P41595,P41972,P42261,P42262,P42330,P42345,P42574,P42684,P42971,P43088,P43115,P43116,P43119,P43133,P43155,P43166,P43681,P43700,P43702,P44345,P44469,P45059,P45954,P46098,P46925,P47712,P47869,P47870,P47901,P47989,P48039,P48048,P48051,P48061,P48169,P48443,P48544,P48549,P48775,P49019,P49146,P49228,P49286,P49327,...,SUB.381,SUB.382,SUB.383,SUB.384,SUB.385,SUB.386,SUB.387,SUB.388,SUB.389,SUB.390,SUB.391,SUB.392,SUB.393,SUB.394,SUB.395,SUB.396,SUB.397,SUB.398,SUB.399,SUB.400,SUB.401,SUB.402,SUB.403,SUB.404,SUB.405,SUB.406,SUB.407,SUB.408,SUB.409,SUB.410,SUB.411,SUB.412,SUB.413,SUB.414,SUB.415,SUB.416,SUB.417,SUB.418,SUB.419,SUB.420,SUB.421,SUB.422,SUB.423,SUB.424,SUB.425,SUB.426,SUB.427,SUB.428,SUB.429,SUB.430,SUB.431,SUB.432,SUB.433,SUB.434,SUB.435,SUB.436,SUB.437,SUB.438,SUB.439,SUB.440,SUB.441,SUB.442,S

In [74]:
drugs_inputs_sider = pd.merge(drugs_inputs, drug_sider, on="Drug", how="left")
drugs_inputs_sider = drugs_inputs_sider.fillna(0)
print(drugs_inputs_sider.shape)
drugs_inputs_sider.head(2)

(944, 3042)


,Drug,A2A2V4,A8MPY1,A9UF02,B8DCL9,B8DD61,C1KC03,O00180,O00305,O00459,O00519,O00555,O00591,O14521,O14646,O14649,O14717,O14732,O14764,O14788,O14920,O15111,O15379,O15399,O15438,O15439,O15530,O15554,O25608,O43451,O43497,O43519,O43526,O43570,O43772,O43923,O60391,O60488,O60494,O60658,O60706,O60733,O60840,O60939,O67135,O75311,O75469,O75493,O75762,O75899,O76074,O76082,O90777,O94956,O95069,O95180,O95263,O95298,O95342,O95477,O95665,P00167,P00183,P00325,P00326,P00363,P00367,P00374,P00390,P00439,P00484,P00491,P00492,P00519,P00533,P00582,P00644,P00720,P00722,P00734,P00742,P00747,P00749,P00750,P00797,P00807,P00811,P00813,P00915,P00918,P01008,P01023,P01130,P01138,P01189,P01375,P01579,P01584,P02359,P02585,P02708,P02787,P02818,P02918,P02919,P03176,P03198,P03200,P03366,P03369,P03372,P03886,P04015,P04035,P04040,P04049,P04054,P04070,P04083,P04150,P04156,P04271,P04293,P04626,P04629,P04746,P04818,P05023,P05091,P05093,P05106,P05107,P05108,P05121,P05164,P05193,P05230,P05231,P05412,P05546,P05981,P06149,P06239,P06241,P06276,P06310,P06401,P06478,P06715,P06746,P06818,P06855,P06864,P06971,P07003,P07098,P07101,P07202,P07225,P07327,P07333,P07437,P07451,P07477,P07510,P07550,P07900,P07947,P07998,P08100,P08149,P08172,P08173,P08174,P08183,P08192,P08235,P08253,P08506,P08514,P08519,P08546,P08588,P08620,P08700,P08729,P08908,P08912,P08913,P09038,P09172,P09210,P09211,P09250,P09252,P09455,P09619,P09884,P09917,P0A0K8,P0A0N4,P0A3M5,P0A3M6,P0A4L9,P0A560,P0A5Y6,P0A674,P0A6E4,P0A6J8,P0A749,P0A7J6,P0A7R5,P0A7S3,P0A7S9,P0A7U3,P0A7V3,P0A7V8,P0A7W7,P0A7X3,P0A7Z4,P0A8T7,P0A8V2,P0AC13,P0AD63,P0AD65,P0AD68,P0ADY7,P0AEB2,P0AEB3,P0AES6,P0AFI2,P0AFI5,P0AG59,P0C002,P0C0X1,P0C0X2,P10109,P10253,P10275,P10276,P10415,P10613,P10696,P10721,P10826,P10827,P10828,P10875,P11137,P11229,P11230,P11387,P11388,P11473,P11485,P11511,P12268,P12314,P12319,P12461,P12821,P12931,P13500,P13569,P13631,P13674,P13716,P13922,P13945,P14324,P14410,P14416,P14555,P14625,P14677,P14679,P14735,P14770,P14780,P14867,P15056,P15121,P15144,P15382,P15538,P15555,P15559,P15692,P16066,P16220,P16233,P16234,P16444,P16502,P16860,P17117,P17302,P17405,P17752,P17787,P17948,P18031,P18089,P18314,P18405,P18505,P18507,P18545,P18825,P18858,P19320,P19634,P19793,P19801,P20061,P20062,P20309,P20594,P20618,P20648,P20701,P20711,P20813,P20839,P21397,P21430,P21462,P21554,P21589,P21728,P21817,P21917,P21918,P21964,P22033,P22102,P22303,P22413,P22748,P23141,P23219,P23258,P23280,P23297,P23415,P23416,P23469,P23786,P23921,P23975,P24024,P24046,P24093,P24228,P24298,P24385,P24530,P24752,P25021,P25024,P25100,P25101,P25103,P25815,P26358,P26676,P27169,P27338,P27361,P27487,P27707,P27815,P27907,P27986,P28074,P28221,P28222,P28223,P28335,P28472,P28476,P28482,P28566,P28702,P29034,P29074,P29166,P29218,P29274,P29275,P29317,P29466,P29474,P29972,P30049,P30085,P30273,P30405,P30518,P30531,P30532,P30536,P30542,P30556,P30926,P30939,P30968,P31213,P31350,P31644,P31645,P31749,P31939,P32239,P32297,P32754,P33151,P33402,P33527,P33673,P33765,P34903,P34969,P34972,P34995,P35218,P35219,P35222,P35228,P35348,P35354,P35367,P35368,P35372,P35408,P35462,P35498,P35499,P35610,P35916,P35968,P36544,P36888,P37088,P37231,P37288,P38435,P38489,P38606,P39086,P39900,P41143,P41145,P41180,P41231,P41594,P41595,P41972,P42261,P42262,P42330,P42345,P42574,P42684,P42971,P43088,P43115,P43116,P43119,P43133,P43155,P43166,P43681,P43700,P43702,P44345,P44469,P45059,P45954,P46098,P46925,P47712,P47869,P47870,P47901,P47989,P48039,P48048,P48051,P48061,P48169,P48443,P48544,P48549,P48775,P49019,P49146,P49228,P49286,P49327,...,trichomonal.vaginitis,trigeminal.neuralgia,tuberculosis,ulcer,loss.of.consciousness,deficiency.anemia,Tympanic.membrane.disorder,erythema,upper.GI.bleeding,upper.respiratory.tract.infection,uremia,calculus.ureteric,ureteral.obstruction,urethritis,urinary.incontinence,urinary.tract.infection,urogenital.anomaly,urticaria,uterine.fibroids,uterine.hemorrhage,uveitis,vaginal.pruritus,vaginitis,varicocele,varicose.vein,Vascular.Disorders,vascular.headache,vasculitis,vasovagal.reaction,venous.insuff

In [75]:
drugs_disease_inputs = pd.merge(drug_disease_flat, drugs_inputs, on="Drug", how="left")
drugs_disease_inputs = drugs_disease_inputs.dropna()
print(drugs_disease_inputs.shape)
drugs_disease_inputs["Target"].value_counts()

(521275, 1659)


0    518373
1      2902
Name: Target, dtype: int64

In [76]:
neg_target = drugs_disease_inputs[drugs_disease_inputs["Target"]==0]
pos_target = drugs_disease_inputs[drugs_disease_inputs["Target"]==1]

pos_target_upsampled = resample(pos_target, replace=True, n_samples=5000, random_state=123) 
neg_target_downsampled = resample(neg_target, replace=False, n_samples=5000, random_state=123) 

drugs_diseases_resampled = pd.concat([neg_target_downsampled, pos_target_upsampled])
drugs_diseases_resampled.to_csv("repurpose_data_flattened_resampled_5000_5000.tsv", sep='\t', index=False)
drugs_diseases_resampled["Target"].value_counts()

0    5000
1    5000
Name: Target, dtype: int64

In [77]:
drugs_disease_inputs_sider = pd.merge(drug_disease_flat, drugs_inputs_sider, on="Drug", how="left")
drugs_disease_inputs_sider = drugs_disease_inputs_sider.dropna()
print(drugs_disease_inputs_sider.shape)
drugs_disease_inputs_sider["Target"].value_counts()

(521275, 3044)


0    518373
1      2902
Name: Target, dtype: int64

In [78]:
neg_target = drugs_disease_inputs_sider[drugs_disease_inputs_sider["Target"]==0]
pos_target = drugs_disease_inputs_sider[drugs_disease_inputs_sider["Target"]==1]

pos_target_upsampled = resample(pos_target, replace=True, n_samples=5000, random_state=123) 
neg_target_downsampled = resample(neg_target, replace=False, n_samples=5000, random_state=123) 

drugs_diseases_resampled_sider = pd.concat([neg_target_downsampled, pos_target_upsampled])
drugs_diseases_resampled_sider.to_csv("repurpose_data_flattened_resampled_sider_5000_5000.tsv", sep='\t', index=False)
drugs_diseases_resampled_sider["Target"].value_counts()

0    5000
1    5000
Name: Target, dtype: int64

In [79]:
drugs_diseases_resampled_sider.shape

(10000, 3044)

# Helper functions

In [116]:
def get_model(model_type, args, random_seed=123):
    if (model_type == "LR"): # Logistic Regression
        model = linear_model.LogisticRegression(C=args[0], solver=args[1][0], penalty=args[1][1], dual=False, tol=0.0001, max_iter=10000, random_state=random_seed)
    elif (model_type == "KNN"):
        model = neighbors.KNeighborsClassifier(n_neighbors=args[0], metric=args[1], weights=args[2])
    elif (model_type == "SVM"): # Support Vector Machine
        model = svm.SVC(kernel=args[0], C=args[1], probability=True)
    elif (model_type == "DT"): # Decision Tree
        model = tree.DecisionTreeClassifier(max_features=args[0], criterion=args[1], min_samples_leaf=args[2], random_state=random_seed)
    elif (model_type == "RF"): # Random Forest
        model = ensemble.RandomForestClassifier(n_estimators=args[0], criterion=args[1], min_samples_leaf=args[2], random_state=random_seed)
    elif (model_type == "GBC"): # Gradient Boosting Classifier
        model = ensemble.GradientBoostingClassifier(n_estimators=args[0], learning_rate=args[1], min_samples_leaf=args[2], loss='deviance',  subsample=1.0, random_state=random_seed)
    elif (model_type == "NN"): # Neural Network
        tf.random.set_seed(random_seed)
        model = Sequential()
        model.add(Dense(args[0], input_dim=78, activation='relu'))
        for j in range(args[2]):
            model.add(Dense(args[1], activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    else:
        print("Model type not supported")
    return model


def stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, model_type, metric = ["F1"], random_seed=123):
    print("Model type: {}".format(model_type))
    top_df = pd.DataFrame([[0, 0, 0, 0]], columns=["TN", "FP", "FN", "TP"])

    for i in range(0, len(parameter_list)):
        print("Expanding parameter: {}".format(parameter_list[i]))
        all_results = []
        model_dict = {}
        for row in top_df.itertuples():
            for x in parameter_dict[parameter_list[i]]:
                args = []
                for j in range(1, i+1):
                    args.append(row[4+j])
                args.append(x)
                args = args + default_args[len(args):]
                print(args)

                pairs = list(zip(parameter_list, args))
                desc = "|".join([x + ":" + str(y) for (x,y) in pairs])
                
                model = get_model(model_type, args, random_seed)
                model_dict[desc] = model
                if model_type in ["LR", "KNN", "SVM", "DT", "RF", "GBC"]:
                    model.fit(x_train.values, y_train)
                    predictions = model.predict(x_test.values)
                    cm = metrics.confusion_matrix(y_test, predictions)
                elif (model_type == "NN"):
                    model.fit(x_train, y_train, epochs=30, batch_size=10, verbose=0)
                    predictions = model.predict(x_test)
                    cm = metrics.confusion_matrix(y_test, np.where(predictions < 0.5, 0, 1))
                else:
                    print("Model type not supported")
                    
                results = cm.ravel().tolist()
                for j in range(0, i+1):
                    results.append(args[j])
                results.append(desc)
                all_results.append(results)

        cols = ["TN", "FP", "FN", "TP"] + parameter_list[:i+1] + ["desc"]
        df = pd.DataFrame(all_results, columns=cols)
        df["correct"] = df["TP"] + df["TN"]
        df["errors"] = df["FP"] + df["FN"]
        df["F1"] = df["TP"] / (df["TP"] + 0.5 * (df["FP"] + df["FN"])) 
        df = df.sort_values(by=[metric[0]], ascending=False)
        top_df = df.head(3)
        print(top_df[["TN", "FP", "FN", "TP"] + parameter_list[:i+1] + [metric[0]]])
        
    df2 = top_df[["desc"] + metric]
    df2 = df2.assign(type=model_type)
    df2 = df2[["type", "desc"] + metric]
    model_desc_filtered = {}
    for desc in df2["desc"]:
        model_desc_filtered[desc] = model_dict[desc]
    return df2, model_desc_filtered


def random_parameter_exploration(parameter_list, parameter_dict, model_type, metric = ["F1"], random_seed=123):
    print("Model type: {}".format(model_type))
    random.seed(123)
    top_df = pd.DataFrame([[0, 0, 0, 0]], columns=["TN", "FP", "FN", "TP"])
    all_results = []
    model_dict = {}
    
    list_of_args = []
    for i in range(0, 20):
        args = []
        for param in parameter_list:
            options = parameter_dict[param]
            args.append(random.choice(options))
        list_of_args.append(args)

    for args in list_of_args:
        print(args)
        pairs = list(zip(parameter_list, args))
        desc = "|".join([x + ":" + str(y) for (x,y) in pairs])

        model = get_model(model_type, args, random_seed)
        model_dict[desc] = model
        if model_type in ["LR", "KNN", "SVM", "DT", "RF", "GBC"]:
            model.fit(x_train.values, y_train)
            predictions = model.predict(x_test.values)
            cm = metrics.confusion_matrix(y_test, predictions)
        elif (model_type == "NN"):
            model.fit(x_train, y_train, epochs=30, batch_size=10, verbose=0)
            predictions = model.predict(x_test)
            cm = metrics.confusion_matrix(y_test, np.where(predictions < 0.5, 0, 1))
        else:
            print("Model type not supported")

        results = cm.ravel().tolist()
        results = results + args
        results.append(desc)
        all_results.append(results)

    cols = ["TN", "FP", "FN", "TP"] + parameter_list + ["desc"]
    df = pd.DataFrame(all_results, columns=cols)
    df["correct"] = df["TP"] + df["TN"]
    df["errors"] = df["FP"] + df["FN"]
    df["F1"] = df["TP"] / (df["TP"] + 0.5 * (df["FP"] + df["FN"])) 
    df = df.sort_values(by=[metric[0]], ascending=False)
    top_df = df.head(3)
    print(top_df[["TN", "FP", "FN", "TP"] + parameter_list + [metric[0]]])
        
    df2 = top_df[["desc"] + metric]
    df2 = df2.assign(type=model_type)
    df2 = df2[["type", "desc"] + metric]
    model_desc_filtered = {}
    for desc in df2["desc"]:
        model_desc_filtered[desc] = model_dict[desc]
    return df2, model_desc_filtered


def stepwise_parameter_exploration_cv(parameter_list, parameter_dict, default_args, model_type, metric = ["f1"], random_seed=123):
    print("Model type: {}".format(model_type))
    top_df = pd.DataFrame([[0]], columns=["f1"])

    for i in range(0, len(parameter_list)):
        print("Expanding parameter: {}".format(parameter_list[i]))
        all_results = []
        model_dict = {}
        for row in top_df.itertuples():
            for x in parameter_dict[parameter_list[i]]:
                args = []
                for j in range(1, i+1):
                    args.append(row[j])
                args.append(x)
                args = args + default_args[len(args):]
                print(args)

                pairs = list(zip(parameter_list, args))
                desc = "|".join([x + ":" + str(y) for (x,y) in pairs])
                
                model = get_model(model_type, args, random_seed)
                model_dict[desc] = model
                if model_type in ["LR", "KNN", "SVM", "DT", "RF", "GBC"]:
                    cv_results = cross_validate(model, features, targets, cv=5, scoring=metric)
                    cv_metrics = [np.mean(cv_results['test_{}'.format(x)]) for x in metric]
                elif (model_type == "NN"):
                    kf = KFold(n_splits=5, shuffle = True, random_state = 123)
                    kf.get_n_splits(features)
                    f1s = []; recalls = []; precisions = []
                    for train_index, test_index in kf.split(features):
                        x_train, x_test = features.iloc[train_index], features.iloc[test_index]
                        y_train, y_test = targets.iloc[train_index], targets.iloc[test_index]
                        model.fit(x_train, y_train, epochs=30, batch_size=10, verbose=0)
                        predictions = model.predict(x_test)
                        cm = metrics.confusion_matrix(y_test, np.where(predictions < 0.5, 0, 1))
                        matrix = cm.ravel().tolist() # ["TN", "FP", "FN", "TP"]
                        f1 = matrix[3] / (matrix[3] + 0.5 * (matrix[1] + matrix[2])); f1s.append(f1)
                        if ((matrix[1] + matrix[3]) > 0):
                            precision = matrix[3] / (matrix[1] + matrix[3])
                            precisions.append(precision)
                        if (matrix[0] + matrix[3] > 0):
                            recall = matrix[3] / (matrix[0] + matrix[3])
                            recalls.append(recall)
                    cv_results = {}
                    cv_results["test_f1"] = np.mean(f1s)
                    cv_results["test_recall"] = np.mean(recall)
                    cv_results["test_precision"] = np.mean(precision)
                    cv_metrics = [cv_results['test_{}'.format(x)] for x in metric]
                else:
                    print("Model type not supported")
                    
                results = []
                for j in range(0, i+1):
                    results.append(args[j])
                results.append(desc)
                results = results + cv_metrics
                all_results.append(results)

        cols = parameter_list[:i+1] + ["desc"] + metric
        df = pd.DataFrame(all_results, columns=cols)
        df = df.sort_values(by=[metric[0]], ascending=False)
        top_df = df.head(3)
        print(top_df[parameter_list[:i+1] + metric])
        
    df2 = top_df[["desc"] + metric]
    df2 = df2.assign(type=model_type)
    df2 = df2[["type", "desc"] + metric]
    model_desc_filtered = {}
    for desc in df2["desc"]:
        model_desc_filtered[desc] = model_dict[desc]
    return df2, model_desc_filtered

# Dimensionality reduction 

In [122]:
# Full data
#drugs_diseases_resampled = pd.read_csv("repurpose_data_flattened_resampled_5000_5000.tsv", sep='\t')
targets = drugs_diseases_resampled['Target'].astype(int)
features = drugs_diseases_resampled.iloc[:, 3:1659].copy()
x_train, x_test, y_train, y_test = train_test_split(features, targets, test_size=0.25, random_state=123)

metric = ["F1"]; random_seed = 123
parameter_list = ["neighbours"]
parameter_dict = {"neighbours": list(range(2,20))}
default_args = [5, "euclidean", "uniform"]
_ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "KNN", metric, random_seed)

Model type: KNN
Expanding parameter: neighbours
[2, 'euclidean', 'uniform']
[3, 'euclidean', 'uniform']
[4, 'euclidean', 'uniform']
[5, 'euclidean', 'uniform']
[6, 'euclidean', 'uniform']
[7, 'euclidean', 'uniform']
[8, 'euclidean', 'uniform']
[9, 'euclidean', 'uniform']
[10, 'euclidean', 'uniform']
[11, 'euclidean', 'uniform']
[12, 'euclidean', 'uniform']
[13, 'euclidean', 'uniform']
[14, 'euclidean', 'uniform']
[15, 'euclidean', 'uniform']
[16, 'euclidean', 'uniform']
[17, 'euclidean', 'uniform']
[18, 'euclidean', 'uniform']
[19, 'euclidean', 'uniform']
     TN   FP   FN   TP  neighbours        F1
15  760  482  444  814          17  0.637431
13  777  465  457  801          15  0.634707
11  785  457  463  795          13  0.633466


In [121]:
metric = ["F1"]; random_seed = 123
parameter_list = ["neighbours"]
parameter_dict = {"neighbours": list(range(2,20))}
default_args = [5, "euclidean", "uniform"]

for reduction in [0.95, 0.9, 0.85, 0.8, 0.75, 0.7]:
    pca = PCA(reduction)
    new_features = pca.fit_transform(features)
    print("For a {} reduction, dataset size is {}.".format(reduction, new_features.shape))

    new_features_df = pd.DataFrame(data = new_features, columns = ["C{}".format(x) for x in range(1, new_features.shape[1] + 1)])
    x_train, x_test, y_train, y_test = train_test_split(new_features_df, targets, test_size=0.25, random_state=123)
    
    _ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "KNN", metric, random_seed)

For a 0.95 reduction, dataset size is (10000, 154).
Model type: KNN
Expanding parameter: neighbours
[2, 'euclidean', 'uniform']
[3, 'euclidean', 'uniform']
[4, 'euclidean', 'uniform']
[5, 'euclidean', 'uniform']
[6, 'euclidean', 'uniform']
[7, 'euclidean', 'uniform']
[8, 'euclidean', 'uniform']
[9, 'euclidean', 'uniform']
[10, 'euclidean', 'uniform']
[11, 'euclidean', 'uniform']
[12, 'euclidean', 'uniform']
[13, 'euclidean', 'uniform']
[14, 'euclidean', 'uniform']
[15, 'euclidean', 'uniform']
[16, 'euclidean', 'uniform']
[17, 'euclidean', 'uniform']
[18, 'euclidean', 'uniform']
[19, 'euclidean', 'uniform']
     TN   FP   FN   TP  neighbours        F1
3   791  451  431  827           5  0.652208
11  761  481  423  835          13  0.648796
5   783  459  434  824           7  0.648564
For a 0.9 reduction, dataset size is (10000, 92).
Model type: KNN
Expanding parameter: neighbours
[2, 'euclidean', 'uniform']
[3, 'euclidean', 'uniform']
[4, 'euclidean', 'uniform']
[5, 'euclidean', 'unifor

# Use more data with dimensionality reduction

In [123]:
neg_target = drugs_disease_inputs[drugs_disease_inputs["Target"]==0]
pos_target = drugs_disease_inputs[drugs_disease_inputs["Target"]==1]

pos_target_upsampled = resample(pos_target, replace=True, n_samples=50000, random_state=123) 
neg_target_downsampled = resample(neg_target, replace=False, n_samples=50000, random_state=123) 

drugs_diseases_resampled_large = pd.concat([neg_target_downsampled, pos_target_upsampled])

targets_large = drugs_diseases_resampled_large['Target'].astype(int)
features_large = drugs_diseases_resampled_large.iloc[:, 3:1659].copy()

In [125]:
metric = ["F1"]; random_seed = 123
parameter_list = ["neighbours"]
parameter_dict = {"neighbours": list(range(2,20))}
default_args = [5, "euclidean", "uniform"]

pca = PCA(0.7)
new_features = pca.fit_transform(features_large)
print("For a {} reduction, dataset size is {}.".format(reduction, new_features.shape))

new_features_df = pd.DataFrame(data = new_features, columns = ["C{}".format(x) for x in range(1, new_features.shape[1] + 1)])
x_train, x_test, y_train, y_test = train_test_split(new_features_df, targets_large, test_size=0.25, random_state=123)

_ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "KNN", metric, random_seed)

For a 0.7 reduction, dataset size is (100000, 22).
Model type: KNN
Expanding parameter: neighbours
[2, 'euclidean', 'uniform']
[3, 'euclidean', 'uniform']
[4, 'euclidean', 'uniform']
[5, 'euclidean', 'uniform']
[6, 'euclidean', 'uniform']
[7, 'euclidean', 'uniform']
[8, 'euclidean', 'uniform']
[9, 'euclidean', 'uniform']
[10, 'euclidean', 'uniform']
[11, 'euclidean', 'uniform']
[12, 'euclidean', 'uniform']
[13, 'euclidean', 'uniform']
[14, 'euclidean', 'uniform']
[15, 'euclidean', 'uniform']
[16, 'euclidean', 'uniform']
[17, 'euclidean', 'uniform']
[18, 'euclidean', 'uniform']
[19, 'euclidean', 'uniform']
      TN    FP    FN    TP  neighbours        F1
15  7888  4760  3949  8403          17  0.658671
17  7918  4730  3978  8374          19  0.657920
13  7908  4740  4018  8334          15  0.655549


# Train models including side effect data 

In [126]:
# Full data
#drugs_diseases_resampled_sider = pd.read_csv("repurpose_data_flattened_resampled_sider_5000_5000.tsv", sep='\t')
targets_sider = drugs_diseases_resampled_sider['Target'].astype(int)
features_sider = drugs_diseases_resampled_sider.iloc[:, 3:].copy()
x_train, x_test, y_train, y_test = train_test_split(features_sider, targets_sider, test_size=0.25, random_state=123)
y_train.value_counts()

metric = ["F1"]; random_seed = 123
parameter_list = ["neighbours"]
parameter_dict = {"neighbours": list(range(2,20))}
default_args = [5, "euclidean", "uniform"]
_ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "KNN", metric, random_seed)

Model type: KNN
Expanding parameter: neighbours
[2, 'euclidean', 'uniform']
[3, 'euclidean', 'uniform']
[4, 'euclidean', 'uniform']
[5, 'euclidean', 'uniform']
[6, 'euclidean', 'uniform']
[7, 'euclidean', 'uniform']
[8, 'euclidean', 'uniform']
[9, 'euclidean', 'uniform']
[10, 'euclidean', 'uniform']
[11, 'euclidean', 'uniform']
[12, 'euclidean', 'uniform']
[13, 'euclidean', 'uniform']
[14, 'euclidean', 'uniform']
[15, 'euclidean', 'uniform']
[16, 'euclidean', 'uniform']
[17, 'euclidean', 'uniform']
[18, 'euclidean', 'uniform']
[19, 'euclidean', 'uniform']
     TN   FP   FN   TP  neighbours        F1
17  769  473  407  851          19  0.659179
15  786  456  418  840          17  0.657792
13  782  460  421  837          15  0.655186


In [127]:
metric = ["F1"]; random_seed = 123
parameter_list = ["neighbours"]
parameter_dict = {"neighbours": list(range(2,20))}
default_args = [5, "euclidean", "uniform"]

for reduction in [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.6, 0.5, 0.4, 0.3]:
    pca = PCA(reduction)
    new_features = pca.fit_transform(features_sider)
    print("For a {} reduction, dataset size is {}.".format(reduction, new_features.shape))

    new_features_df = pd.DataFrame(data = new_features, columns = ["C{}".format(x) for x in range(1, new_features.shape[1] + 1)])
    x_train, x_test, y_train, y_test = train_test_split(new_features_df, targets_sider, test_size=0.25, random_state=123)
    
    _ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "KNN", metric, random_seed)

For a 0.95 reduction, dataset size is (10000, 287).
Model type: KNN
Expanding parameter: neighbours
[2, 'euclidean', 'uniform']
[3, 'euclidean', 'uniform']
[4, 'euclidean', 'uniform']
[5, 'euclidean', 'uniform']
[6, 'euclidean', 'uniform']
[7, 'euclidean', 'uniform']
[8, 'euclidean', 'uniform']
[9, 'euclidean', 'uniform']
[10, 'euclidean', 'uniform']
[11, 'euclidean', 'uniform']
[12, 'euclidean', 'uniform']
[13, 'euclidean', 'uniform']
[14, 'euclidean', 'uniform']
[15, 'euclidean', 'uniform']
[16, 'euclidean', 'uniform']
[17, 'euclidean', 'uniform']
[18, 'euclidean', 'uniform']
[19, 'euclidean', 'uniform']
     TN   FP   FN   TP  neighbours        F1
17  816  426  441  817          19  0.653339
15  806  436  442  816          17  0.650199
13  802  440  450  808          15  0.644852
For a 0.9 reduction, dataset size is (10000, 196).
Model type: KNN
Expanding parameter: neighbours
[2, 'euclidean', 'uniform']
[3, 'euclidean', 'uniform']
[4, 'euclidean', 'uniform']
[5, 'euclidean', 'unifo

#  Stepwise Parameter Exploration

In [128]:
targets_sider = drugs_diseases_resampled_sider['Target'].astype(int)
features_sider = drugs_diseases_resampled_sider.iloc[:, 3:].copy()

reduction = 0.75
pca = PCA(reduction)
new_features = pca.fit_transform(features_sider)
print("For a {} reduction, dataset size is {}.".format(reduction, new_features.shape))

new_features_df = pd.DataFrame(data = new_features, columns = ["C{}".format(x) for x in range(1, new_features.shape[1] + 1)])
x_train, x_test, y_train, y_test = train_test_split(new_features_df, targets_sider, test_size=0.25, random_state=123)

For a 0.75 reduction, dataset size is (10000, 78).


In [110]:
metric = ["F1"]
random_seed = 123
best_candidates = pd.DataFrame([], columns=["type", "desc"] + metric)

start_time = datetime.now()
parameter_list = ["C", "solver"]
parameter_dict = {"C": [1, 2, 5, 10, 20, 50, 100, 200, 500],
                 "solver": [('newton-cg', 'l2'), ('lbfgs', 'l2'), ('liblinear', 'l2'), ('sag', 'l2'), ('saga', 'l2')]}
default_args = [1, ('newton-cg', 'l2')]
new_candidates, _ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "LR", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("LR", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["neighbours", "dist_type", "weights"]
parameter_dict = {"neighbours": list(range(2,20)), 
                  "dist_type": ["euclidean", "manhattan", "chebyshev", "minkowski"], 
                  "weights": ["uniform", "distance"]}
default_args = [5, "euclidean", "uniform"]
new_candidates, _ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "KNN", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("KNN", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["kernel", "C"]
parameter_dict = {"kernel": ["linear", "poly", "rbf", "sigmoid"], 
                  "C": [1, 2, 5, 10, 20]}
default_args = ["linear", 1]
new_candidates, _ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "SVM", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("SVM", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["max_features", "criterion", "min_samples_leaf"]
parameter_dict = {"max_features": [5, 10, 20, 30, 40], 
                  "criterion":["gini", "entropy"],
                 "min_samples_leaf": [1,3,5,10,15]}
default_args = [40, "gini", 1]
new_candidates, _ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "DT", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("DT", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["n_estimators", "criterion", "min_samples_leaf"]
parameter_dict = {"n_estimators": [5, 10, 20, 30, 40, 50], 
                  "criterion":["gini", "entropy"],
                 "min_samples_leaf": [1, 3, 5, 10, 15]}
default_args = [10, "gini", 1]
new_candidates, _ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "RF", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("RF", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["n_est", "learn_rate", "min_samp_leaf"]
parameter_dict = {"n_est": [5, 10, 20, 30, 40, 50], 
                  "learn_rate": [0.1, 0.15, 0.2, 0.25, 0.3], 
                 "min_samp_leaf": [1, 3, 5, 10, 15]}
default_args = [10, 0.1, 1]
new_candidates, _ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "GBC", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("GBC", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["1st_lay_nodes", "hid_lay_nodes", "hid_layers"]
parameter_dict = {"1st_lay_nodes": [5, 10, 15, 20, 25, 30, 35, 40], 
                    "hid_lay_nodes": [5, 10, 15, 20, 25, 30, 35, 40], 
                    "hid_layers": [0, 1, 2, 3, 4, 6, 8]}
default_args = [10, 10, 0]
new_candidates, _ = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "NN", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("NN", (end_time - start_time)))

Model type: LR
Expanding parameter: C
[1, ('newton-cg', 'l2')]
[2, ('newton-cg', 'l2')]
[5, ('newton-cg', 'l2')]
[10, ('newton-cg', 'l2')]
[20, ('newton-cg', 'l2')]
[50, ('newton-cg', 'l2')]
[100, ('newton-cg', 'l2')]
[200, ('newton-cg', 'l2')]
[500, ('newton-cg', 'l2')]
    TN   FP   FN   TP  C        F1
0  810  432  552  706  1  0.589316
1  810  432  552  706  2  0.589316
2  810  432  552  706  5  0.589316
Expanding parameter: solver
[1, ('newton-cg', 'l2')]
[1, ('lbfgs', 'l2')]
[1, ('liblinear', 'l2')]
[1, ('sag', 'l2')]
[1, ('saga', 'l2')]
[2, ('newton-cg', 'l2')]
[2, ('lbfgs', 'l2')]
[2, ('liblinear', 'l2')]
[2, ('sag', 'l2')]
[2, ('saga', 'l2')]
[5, ('newton-cg', 'l2')]
[5, ('lbfgs', 'l2')]
[5, ('liblinear', 'l2')]
[5, ('sag', 'l2')]
[5, ('saga', 'l2')]
    TN   FP   FN   TP  C           solver        F1
0  810  432  552  706  1  (newton-cg, l2)  0.589316
1  810  432  552  706  1      (lbfgs, l2)  0.589316
2  810  432  552  706  1  (liblinear, l2)  0.589316
Elapsed duration to tr

[40, 5, 1]
[40, 5, 2]
[40, 5, 3]
[40, 5, 4]
[40, 5, 6]
[40, 5, 8]
[40, 10, 0]
[40, 10, 1]
[40, 10, 2]
[40, 10, 3]
[40, 10, 4]
[40, 10, 6]
[40, 10, 8]
[40, 15, 0]
[40, 15, 1]
[40, 15, 2]
[40, 15, 3]
[40, 15, 4]
[40, 15, 6]
[40, 15, 8]
     TN   FP   FN   TP  1st_lay_nodes  hid_lay_nodes  hid_layers        F1
19  751  491  385  873             40             15           6  0.665904
2   740  502  382  876             40              5           2  0.664643
16  759  483  399  859             40             15           2  0.660769
Elapsed duration to train NN is 0:27:39.302857.


In [111]:
best_candidates = best_candidates.sort_values(by=['F1'], ascending=False)
print(best_candidates.to_string(index=False))

type                                                  desc       F1
 KNN    neighbours:11|dist_type:chebyshev|weights:distance 0.687060
 KNN    neighbours:13|dist_type:manhattan|weights:distance 0.683491
 KNN    neighbours:11|dist_type:manhattan|weights:distance 0.683252
  NN        1st_lay_nodes:40|hid_lay_nodes:15|hid_layers:6 0.665904
  NN         1st_lay_nodes:40|hid_lay_nodes:5|hid_layers:2 0.664643
  NN        1st_lay_nodes:40|hid_lay_nodes:15|hid_layers:2 0.660769
  RF  n_estimators:10|criterion:entropy|min_samples_leaf:5 0.652225
 SVM                                      kernel:poly|C:20 0.652003
 SVM                                       kernel:rbf|C:20 0.650706
  RF     n_estimators:10|criterion:gini|min_samples_leaf:1 0.648840
  RF  n_estimators:10|criterion:entropy|min_samples_leaf:1 0.648840
 SVM                                       kernel:rbf|C:10 0.645340
  DT max_features:40|criterion:entropy|min_samples_leaf:15 0.643026
  DT     max_features:5|criterion:gini|min_sampl

# Random Parameter Exploration

In [112]:
metric = ["F1"]
best_candidates = pd.DataFrame([], columns=["type", "desc"] + metric)


parameter_list = ["C", "solver"]
parameter_dict = {"C": [1, 2, 5, 10, 20, 50, 100, 200, 500],
                 "solver": [('newton-cg', 'l2'), ('lbfgs', 'l2'), ('liblinear', 'l2'), ('sag', 'l2'), ('saga', 'l2')]}
new_candidates, _ = random_parameter_exploration(parameter_list, parameter_dict, "LR", metric)
best_candidates = pd.concat([best_candidates, new_candidates])


parameter_list = ["neighbours", "dist_type", "weights"]
parameter_dict = {"neighbours": list(range(2,20)), 
                  "dist_type": ["euclidean", "manhattan", "chebyshev", "minkowski"], 
                  "weights": ["uniform", "distance"]}
new_candidates, _  = random_parameter_exploration(parameter_list, parameter_dict, "KNN", metric)
best_candidates = pd.concat([best_candidates, new_candidates])


parameter_list = ["kernel", "C"]
parameter_dict = {"kernel": ["linear", "poly", "rbf", "sigmoid"], 
                  "C": [1, 2, 5, 10, 20]}
new_candidates, _  = random_parameter_exploration(parameter_list, parameter_dict, "SVM", metric)
best_candidates = pd.concat([best_candidates, new_candidates])


parameter_list = ["max_features", "criterion", "min_samples_leaf"]
parameter_dict = {"max_features": [5, 10, 20, 30, 40], 
                  "criterion":["gini", "entropy"],
                 "min_samples_leaf": [1,3,5,10,15]}
new_candidate, _ = random_parameter_exploration(parameter_list, parameter_dict, "DT", metric)
best_candidates = pd.concat([best_candidates, new_candidates])


parameter_list = ["n_estimators", "criterion", "min_samples_leaf"]
parameter_dict = {"n_estimators": [5, 10, 20, 30, 40, 50], 
                  "criterion":["gini", "entropy"],
                 "min_samples_leaf": [1, 3, 5, 10, 15]}
new_candidates, _  = random_parameter_exploration(parameter_list, parameter_dict, "RF", metric)
best_candidates = pd.concat([best_candidates, new_candidates])


parameter_list = ["n_est", "learn_rate", "min_samp_leaf"]
parameter_dict = {"n_est": [5, 10, 20, 30, 40, 50], 
                  "learn_rate": [0.1, 0.15, 0.2, 0.25, 0.3], 
                 "min_samp_leaf": [1, 3, 5, 10, 15]}
new_candidates, _  = random_parameter_exploration(parameter_list, parameter_dict, "GBC", metric)
best_candidates = pd.concat([best_candidates, new_candidates])


parameter_list = ["1st_lay_nodes", "hid_lay_nodes", "hid_layers"]
parameter_dict = {"1st_lay_nodes": [5, 10, 15, 20, 25, 30, 35, 40], 
                    "hid_lay_nodes": [5, 10, 15, 20, 25, 30, 35, 40], 
                    "hid_layers": [0, 1, 2, 3, 4, 6, 8]}
new_candidates, _  = random_parameter_exploration(parameter_list, parameter_dict, "NN", metric)
best_candidates = pd.concat([best_candidates, new_candidates])

Model type: LR
[1, ('liblinear', 'l2')]
[2, ('sag', 'l2')]
[20, ('newton-cg', 'l2')]
[1, ('sag', 'l2')]
[500, ('saga', 'l2')]
[50, ('liblinear', 'l2')]
[1, ('lbfgs', 'l2')]
[5, ('liblinear', 'l2')]
[500, ('liblinear', 'l2')]
[10, ('lbfgs', 'l2')]
[1, ('sag', 'l2')]
[2, ('saga', 'l2')]
[100, ('newton-cg', 'l2')]
[1, ('liblinear', 'l2')]
[200, ('newton-cg', 'l2')]
[1, ('newton-cg', 'l2')]
[5, ('lbfgs', 'l2')]
[1, ('liblinear', 'l2')]
[100, ('saga', 'l2')]
[200, ('liblinear', 'l2')]
     TN   FP   FN   TP    C           solver        F1
0   810  432  552  706    1  (liblinear, l2)  0.589316
1   810  432  552  706    2        (sag, l2)  0.589316
18  810  432  552  706  100       (saga, l2)  0.589316
Model type: KNN
[3, 'chebyshev', 'uniform']
[15, 'chebyshev', 'uniform']
[3, 'minkowski', 'distance']
[12, 'euclidean', 'uniform']
[6, 'chebyshev', 'distance']
[9, 'manhattan', 'uniform']
[15, 'euclidean', 'distance']
[4, 'euclidean', 'distance']
[16, 'euclidean', 'uniform']
[4, 'manhattan', 'u

In [113]:
best_candidates = best_candidates.sort_values(by=['F1'], ascending=False)
print(best_candidates.to_string(index=False))

type                                                 desc       F1
 KNN    neighbours:7|dist_type:chebyshev|weights:distance 0.689175
 KNN   neighbours:17|dist_type:chebyshev|weights:distance 0.686599
 KNN    neighbours:6|dist_type:chebyshev|weights:distance 0.673091
  NN       1st_lay_nodes:40|hid_lay_nodes:20|hid_layers:8 0.668874
  NN       1st_lay_nodes:40|hid_lay_nodes:25|hid_layers:3 0.661333
  NN       1st_lay_nodes:30|hid_lay_nodes:30|hid_layers:6 0.660000
  RF    n_estimators:40|criterion:gini|min_samples_leaf:3 0.648733
  RF n_estimators:20|criterion:entropy|min_samples_leaf:3 0.648206
  RF n_estimators:20|criterion:entropy|min_samples_leaf:3 0.648206
 SVM                                      kernel:poly|C:5 0.643317
 SVM                                      kernel:poly|C:5 0.643317
 SVM                                       kernel:rbf|C:5 0.641797
 SVM                                       kernel:rbf|C:5 0.641797
 SVM                                       kernel:rbf|C:5 0.64

# Cross validation

In [131]:
metric = ["f1"]
best_candidates = pd.DataFrame([], columns=["type", "desc"] + metric)


start_time = datetime.now()
parameter_list = ["C", "solver"]
parameter_dict = {"C": [1, 2, 5, 10, 20, 50, 100, 200, 500],
                 "solver": [('newton-cg', 'l2'), ('lbfgs', 'l2'), ('liblinear', 'l2'), ('sag', 'l2'), ('saga', 'l2')]}
default_args = [1, ('newton-cg', 'l2')]
new_candidates, _ = stepwise_parameter_exploration_cv(parameter_list, parameter_dict, default_args, "LR", metric)
best_candidates = pd.concat([best_candidates, new_candidates])
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("LR", (end_time - start_time)))

Model type: LR
Expanding parameter: C
[1, ('newton-cg', 'l2')]
[2, ('newton-cg', 'l2')]
[5, ('newton-cg', 'l2')]
[10, ('newton-cg', 'l2')]
[20, ('newton-cg', 'l2')]
[50, ('newton-cg', 'l2')]
[100, ('newton-cg', 'l2')]
[200, ('newton-cg', 'l2')]
[500, ('newton-cg', 'l2')]
     C        f1
3   10  0.645648
7  200  0.645412
1    2  0.645326
Expanding parameter: solver
[10, ('newton-cg', 'l2')]
[10, ('lbfgs', 'l2')]
[10, ('liblinear', 'l2')]
[10, ('sag', 'l2')]
[10, ('saga', 'l2')]
[200, ('newton-cg', 'l2')]
[200, ('lbfgs', 'l2')]
[200, ('liblinear', 'l2')]
[200, ('sag', 'l2')]
[200, ('saga', 'l2')]
[2, ('newton-cg', 'l2')]
[2, ('lbfgs', 'l2')]
[2, ('liblinear', 'l2')]
[2, ('sag', 'l2')]
[2, ('saga', 'l2')]
    C           solver        f1
1  10      (lbfgs, l2)  0.646194
0  10  (newton-cg, l2)  0.645648
2  10  (liblinear, l2)  0.645648
Elapsed duration to train LR is 2:38:38.665714.


In [132]:
best_candidates_cv = best_candidates.sort_values(by=['f1'], ascending=False)
print(best_candidates_cv.to_string(index=False))

type                            desc       f1
  LR     C:10|solver:('lbfgs', 'l2') 0.646194
  LR C:10|solver:('newton-cg', 'l2') 0.645648
  LR C:10|solver:('liblinear', 'l2') 0.645648


# Final evaluation datasets

In [133]:
targets_sider = drugs_diseases_resampled_sider['Target'].astype(int)
features_sider = drugs_diseases_resampled_sider.iloc[:, 3:].copy()

reduction = 0.75
pca = PCA(reduction)
new_features = pca.fit_transform(features_sider)
print("For a {} reduction, dataset size is {}.".format(reduction, new_features.shape))

new_features_df = pd.DataFrame(data = new_features, columns = ["C{}".format(x) for x in range(1, new_features.shape[1] + 1)])
x_traintest, x_test_final, y_traintest, y_test_final = train_test_split(new_features_df, targets_sider, test_size=0.2, random_state=123)
x_train, x_test, y_train, y_test = train_test_split(x_traintest, y_traintest, test_size=0.25, random_state=123)

For a 0.75 reduction, dataset size is (10000, 78).


In [134]:
metric = ["F1"]
random_seed = 321
best_candidates = pd.DataFrame([], columns=["type", "desc"] + metric)
mixed_model_dict = {}

start_time = datetime.now()
parameter_list = ["C", "solver"]
parameter_dict = {"C": [1, 2, 5, 10, 20, 50, 100, 200, 500],
                 "solver": [('newton-cg', 'l2'), ('lbfgs', 'l2'), ('liblinear', 'l2'), ('sag', 'l2'), ('saga', 'l2')]}
default_args = [1, ('newton-cg', 'l2')]
new_candidates, model_dict = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "LR", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
mixed_model_dict["LR"] = model_dict
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("LR", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["neighbours", "dist_type", "weights"]
parameter_dict = {"neighbours": list(range(2,20)), 
                  "dist_type": ["euclidean", "manhattan", "chebyshev", "minkowski"], 
                  "weights": ["uniform", "distance"]}
default_args = [5, "euclidean", "uniform"]
new_candidates, model_dict = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "KNN", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
mixed_model_dict["KNN"] = model_dict
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("KNN", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["max_features", "criterion", "min_samples_leaf"]
parameter_dict = {"max_features": [5, 10, 20, 30, 40], 
                  "criterion":["gini", "entropy"],
                 "min_samples_leaf": [1,3,5,10,15]}
default_args = [40, "gini", 1]
new_candidates, model_dict = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "DT", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
mixed_model_dict["DT"] = model_dict
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("DT", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["n_estimators", "criterion", "min_samples_leaf"]
parameter_dict = {"n_estimators": [5, 10, 20, 30, 40, 50], 
                  "criterion":["gini", "entropy"],
                 "min_samples_leaf": [1, 3, 5, 10, 15]}
default_args = [10, "gini", 1]
new_candidates, model_dict = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "RF", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
mixed_model_dict["RF"] = model_dict
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("RF", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["n_est", "learn_rate", "min_samp_leaf"]
parameter_dict = {"n_est": [5, 10, 20, 30, 40, 50], 
                  "learn_rate": [0.1, 0.15, 0.2, 0.25, 0.3], 
                 "min_samp_leaf": [1, 3, 5, 10, 15]}
default_args = [10, 0.1, 1]
new_candidates, model_dict = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "GBC", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
mixed_model_dict["GBC"] = model_dict
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("GBC", (end_time - start_time)))


start_time = datetime.now()
parameter_list = ["1st_lay_nodes", "hid_lay_nodes", "hid_layers"]
parameter_dict = {"1st_lay_nodes": [5, 10, 15, 20, 25, 30, 35, 40], 
                    "hid_lay_nodes": [5, 10, 15, 20, 25, 30, 35, 40], 
                    "hid_layers": [0, 1, 2, 3, 4, 6, 8]}
default_args = [10, 10, 0]
new_candidates, model_dict = stepwise_parameter_exploration(parameter_list, parameter_dict, default_args, "NN", metric, random_seed)
best_candidates = pd.concat([best_candidates, new_candidates])
mixed_model_dict["NN"] = model_dict
end_time = datetime.now()
print("Elapsed duration to train {} is {}.".format("NN", (end_time - start_time)))

Model type: LR
Expanding parameter: C
[1, ('newton-cg', 'l2')]
[2, ('newton-cg', 'l2')]
[5, ('newton-cg', 'l2')]
[10, ('newton-cg', 'l2')]
[20, ('newton-cg', 'l2')]
[50, ('newton-cg', 'l2')]
[100, ('newton-cg', 'l2')]
[200, ('newton-cg', 'l2')]
[500, ('newton-cg', 'l2')]
    TN   FP   FN   TP  C        F1
0  642  354  466  538  1  0.567511
1  642  354  466  538  2  0.567511
2  642  354  466  538  5  0.567511
Expanding parameter: solver
[1, ('newton-cg', 'l2')]
[1, ('lbfgs', 'l2')]
[1, ('liblinear', 'l2')]
[1, ('sag', 'l2')]
[1, ('saga', 'l2')]
[2, ('newton-cg', 'l2')]
[2, ('lbfgs', 'l2')]
[2, ('liblinear', 'l2')]
[2, ('sag', 'l2')]
[2, ('saga', 'l2')]
[5, ('newton-cg', 'l2')]
[5, ('lbfgs', 'l2')]
[5, ('liblinear', 'l2')]
[5, ('sag', 'l2')]
[5, ('saga', 'l2')]
    TN   FP   FN   TP  C           solver        F1
0  642  354  466  538  1  (newton-cg, l2)  0.567511
1  642  354  466  538  1      (lbfgs, l2)  0.567511
2  642  354  466  538  1  (liblinear, l2)  0.567511
Elapsed duration to tr

In [135]:
def parse_args(desc):
    args = []
    arg_vals = desc.split("|")
    for arg_val in arg_vals:
        val = arg_val.split(":")[1]
        if val.startswith("("):
            val = val[1:-1]
            val = val.replace("'", "").replace(" ", "")
            val = tuple(val.split(","))
        args.append(val)
    return args

metric=["F1"]
all_results=[]
top_df = pd.DataFrame([[0, 0, 0, 0]], columns=["TN", "FP", "FN", "TP"])
for model_type in mixed_model_dict.keys():
    for desc in mixed_model_dict[model_type].keys():
        args = parse_args(desc)
        model = mixed_model_dict[model_type][desc]
        if model_type in ["LR", "KNN", "SVM", "DT", "RF", "GBC"]:
            predictions = model.predict(x_test_final.values)
            cm = metrics.confusion_matrix(y_test_final, predictions)
        elif (model_type == "NN"):
            predictions = model.predict(x_test_final)
            cm = metrics.confusion_matrix(y_test_final, np.where(predictions < 0.5, 0, 1))
        else:
            print("Model type not supported")
            
        results = cm.ravel().tolist()
        results.append(model_type)
        results.append(desc)
        all_results.append(results)

cols = ["TN", "FP", "FN", "TP", "type", "desc"]
df = pd.DataFrame(all_results, columns=cols)
df["correct"] = df["TP"] + df["TN"]
df["errors"] = df["FP"] + df["FN"]
df["F1"] = df["TP"] / (df["TP"] + 0.5 * (df["FP"] + df["FN"])) 
df = df.sort_values(by=[metric[0]], ascending=False)
print(df[["type", "desc"] + [metric[0]]].to_string(index=False))

type                                                  desc       F1
 KNN     neighbours:7|dist_type:manhattan|weights:distance 0.677605
 KNN     neighbours:7|dist_type:chebyshev|weights:distance 0.674720
 KNN     neighbours:5|dist_type:chebyshev|weights:distance 0.673524
  RF  n_estimators:20|criterion:entropy|min_samples_leaf:1 0.650438
  RF     n_estimators:20|criterion:gini|min_samples_leaf:1 0.650122
  NN        1st_lay_nodes:40|hid_lay_nodes:10|hid_layers:1 0.648087
  NN        1st_lay_nodes:40|hid_lay_nodes:15|hid_layers:8 0.646977
  NN        1st_lay_nodes:40|hid_lay_nodes:10|hid_layers:0 0.644737
  RF n_estimators:20|criterion:entropy|min_samples_leaf:10 0.642111
 GBC              n_est:50|learn_rate:0.3|min_samp_leaf:15 0.640536
 GBC              n_est:50|learn_rate:0.3|min_samp_leaf:10 0.635184
  DT    max_features:40|criterion:gini|min_samples_leaf:10 0.634294
  DT    max_features:20|criterion:gini|min_samples_leaf:10 0.632860
  DT     max_features:5|criterion:gini|min_sampl